In [ ]:
pip install pyswarms

In [ ]:
pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import kstest
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from pyswarms.discrete import BinaryPSO
from pyswarms.utils.plotters import plot_cost_history
import matplotlib.pyplot as plt

In [ ]:
dataset=pd.read_csv('/content/cervical-cancer_csv.csv')

In [ ]:
dataset.columns

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.nunique()

In [ ]:
dataset.hist(grid=False,figsize=(20,20))
plt.show()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset = pd.get_dummies(dataset, drop_first=True)

In [ ]:
numerical_columns = dataset.select_dtypes(include=np.number).columns
categorical_columns = dataset.select_dtypes(include='object').columns

# Filling missing values in numeric variables with median
for column in numerical_columns:
    mean_value = dataset[column].mean()
    dataset.loc[:, column] = dataset[column].fillna(mean_value)

# Finally, convert all data to numeric values
    dataset = dataset.apply(pd.to_numeric)

In [ ]:
dataset.isna().sum()

In [ ]:
value_counts = dataset['Dx:Cancer'].value_counts()

print(value_counts)

In [ ]:
dataset.drop(['STDs: Time since first diagnosis', 'STDs: Time since last diagnosis','STDs:cervical condylomatosis','STDs:AIDS'], axis=1, inplace=True)
# cols_to_use = ['Age','Num of pregnancies','Dx:Cancer', 'Smokes', 'Smokes (years)', 'Smokes (packs/year)','IUD','IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis','STDs:vulvo-perineal condylomatosis','STDs:HIV','STDs: Number of diagnosis','STDs: Time since first diagnosis', 'STDs: Time since last diagnosis','Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx','Hinselmann','Schiller','Biopsy']
# dataset = dataset[cols_to_use]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming the last column is the target, select all columns except the last one
features = dataset.columns[:-1]

plt.figure(figsize=(15,8))
sns.boxplot(data=dataset[features], orient='h')
plt.title('Boxplot of Features in Whole Dataset')
plt.xlabel('Feature Value')
plt.show()


In [ ]:
columns_to_winsorize = ['Smokes', 'Smokes (years)', 'Smokes (packs/year)', 'IUD (years)']
lower_percentile = 0.05
upper_percentile = 0.95

for column in columns_to_winsorize:
    lower_bound = dataset[column].quantile(lower_percentile)
    upper_bound = dataset[column].quantile(upper_percentile)
    dataset['winsorized_' + column] = dataset[column].clip(lower=lower_bound, upper=upper_bound)

print("\nOriginal Data:")
print(dataset[columns_to_winsorize].head())

print("\nWinsorized Data:")
print(dataset[['winsorized_' + col for col in columns_to_winsorize]].head())

In [ ]:

import matplotlib.pyplot as plt

counts = dataset['Biopsy'].value_counts().sort_index()

plt.bar(counts.index, counts.values, color=['green', 'red'])
plt.xticks([0, 1], ['No(0)', 'Yes (1)'])
plt.xlabel('Biopsy')
plt.ylabel('Count')
plt.title('Distribution of Biopsy')
plt.show()


In [ ]:
corr_matrix = dataset.corr()

# Plot heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
dataset.columns

In [ ]:
print(type(dataset))  # This will print the actual type of `dataset`


In [ ]:
features = ['Age',
           'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Smokes (packs/year)',
            'IUD',
           'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
           'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis',
           'STDs:syphilis',
           'STDs: Number of diagnosis', 'Dx:CIN', 'Dx', 'Hinselmann',
           'Schiller']

target = ['Dx:Cancer','Biopsy']

In [ ]:


# Encode categorical columns
for col in dataset.columns:
    if dataset[col].dtype == 'object':
        dataset[col] = LabelEncoder().fit_transform(dataset[col].astype(str))

X = dataset[features].values
y = dataset[target].values

In [ ]:
import matplotlib.pyplot as plt

counts = dataset['Dx:Cancer'].value_counts().sort_index()

plt.bar(counts.index, counts.values, color=['green', 'red'])
plt.xticks([0, 1], ['No Cancer (0)', 'Cancer (1)'])
plt.xlabel('Dx:Cancer')
plt.ylabel('Count')
plt.title('Distribution of Dx:Cancer')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder

# If needed, convert y_train to 1D
y_train_single = np.argmax(y_train, axis=1) if y_train.ndim > 1 else y_train

smote_enn = SMOTEENN(random_state=42)
X_train_res, y_train_res = smote_enn.fit_resample(X_train, y_train_single)


print("After SMOTEENN:")
print("X_train_res shape:", X_train_res.shape)
print("y_train_res distribution:", np.bincount(y_train_res))


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Initialize base classifier
logreg = LogisticRegression(random_state=42, max_iter=1000)

# Wrap it in MultiOutputClassifier
model = MultiOutputClassifier(logreg)
model.fit(X_train, y_train)

# Predict on train and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluate overall accuracy
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# List of target columns
target_cols = ['Dx:Cancer', 'Biopsy']  # Replace with your actual target names if different

# Print results
print(f"\nTraining Accuracy: {train_acc:.4f} | Testing Accuracy: {test_acc:.4f}")

# Print confusion matrices for each target column
for i, col in enumerate(target_cols):
    print(f"\nConfusion Matrix for '{col}':")
    print(confusion_matrix(y_test[:, i], y_test_pred[:, i]))


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Initialize base classifier
tree_model = DecisionTreeClassifier(random_state=42)


forest = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=42
)
model = MultiOutputClassifier(forest)
model.fit(X_train, y_train)


# Wrap in MultiOutputClassifier for multi-label support
model = MultiOutputClassifier(tree_model)
model.fit(X_train, y_train)

# Predict on train and test
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Accuracy scores
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

# Define target columns
target_cols = ['Dx:Cancer', 'Biopsy']  # Adjust based on your actual target column names

# Print overall accuracy
print(f"\nTraining Accuracy: {train_acc:.4f} | Testing Accuracy: {test_acc:.4f}")

# Print confusion matrix and classification report for each target
for i, col in enumerate(target_cols):
    print(f"\nConfusion Matrix for '{col}':")
    print(confusion_matrix(y_test[:, i], y_test_pred[:, i]))

    print(f"\nClassification Report for '{col}':")
    print(classification_report(y_test[:, i], y_test_pred[:, i], zero_division=0))


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.multioutput import MultiOutputClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=8,                    # restrict tree depth
    min_samples_split=10,           # avoid small splits
    min_samples_leaf=5,             # more samples per leaf
    class_weight='balanced',        # handle class imbalance
    random_state=42
)

model = MultiOutputClassifier(rf)
model.fit(X_train, y_train)


y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

target_cols = ['Dx:Cancer', 'Biopsy']  # replace with your actual target names

print(f"\nTraining Accuracy: {train_acc:.4f} | Testing Accuracy: {test_acc:.4f}")

for i, col in enumerate(target_cols):
    print(f"\nConfusion Matrix for '{col}':")
    print(confusion_matrix(y_test[:, i], y_test_pred[:, i]))



# Xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_estimators=100,
    max_depth=4,              # lower depth = less overfitting
    learning_rate=0.1,        # slower learning = better generalization
    subsample=0.8,            # only use 80% of rows per tree
    colsample_bytree=0.8,     # only use 80% of columns per tree
    reg_alpha=0.5,            # L1 regularization
    reg_lambda=1.0,           # L2 regularization
    scale_pos_weight=5        # adjust based on your class imbalance
)


model = MultiOutputClassifier(xgb)
model.fit(X_train, y_train)


y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

target_cols = ['Dx:Cancer', 'Biopsy']  # replace with your actual target names

print(f"\nTraining Accuracy: {train_acc:.4f} | Testing Accuracy: {test_acc:.4f}")

for i, col in enumerate(target_cols):
    print(f"\nConfusion Matrix for '{col}':")
    print(confusion_matrix(y_test[:, i], y_test_pred[:, i]))


In [ ]:
pip install --upgrade catboost


In [ ]:
pip install lightgbm

# Catboost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

catboost_model = CatBoostClassifier(
    random_seed=42,
    iterations=100,
    depth=4,
    learning_rate=0.1,
    verbose=False
)



# Wrap in MultiOutputClassifier
model = MultiOutputClassifier(catboost_model)

# Train the model
model.fit(X_train, y_train)

# Predict
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Accuracy scores
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"\nTraining Accuracy: {train_acc:.4f} | Testing Accuracy: {test_acc:.4f}")

target_cols = ['Dx:Cancer', 'Biopsy']

for i, col in enumerate(target_cols):
    print(f"\nConfusion Matrix for '{col}':")
    print(confusion_matrix(y_test[:, i], y_test_pred[:, i]))

    print(f"\nClassification Report for '{col}':")
    print(classification_report(y_test[:, i], y_test_pred[:, i], zero_division=0))


In [ ]:
import pandas as pd

# Prepare the data
results = {
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest', 'XGBoost', 'CatBoost'],
    'Training Accuracy': [0.9476, 0.9940, 0.9521, 0.9775, 0.9671],
    'Testing Accuracy': [0.9461, 0.9461, 0.9581, 0.9581, 0.9581]
}

# Create DataFrame
df_results = pd.DataFrame(results)

# Display the table
print(df_results)


# PSO-feature extraction

In [ ]:
pip install catboost pyswarms scikit-learn


In [ ]:
pip install optuna

In [ ]:
pip install -U imbalanced-learn


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assume these are predefined:
# X_train, y_train, X_test, y_test: your data arrays
# features: list of feature names
# targets: list of target names (length = y_train.shape[1])
# best_particle: result from your PSO optimization (array of shape [n_features])

def objective_function(particles):
    scores = []
    for particle in particles:
        mask = particle > 0.5
        if np.sum(mask) == 0:
            scores.append(1.0)  # Penalize if no features selected
            continue
        X_sel = X_train[:, mask]

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        accs = []

        # We'll do CV based on the first target stratification as proxy
        for train_idx, val_idx in skf.split(X_sel, y_train[:, 0]):
            X_tr, X_val = X_sel[train_idx], X_sel[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            acc_list = []
            for i in range(y_train.shape[1]):
                # Apply SMOTE on training fold per target
                smote = SMOTE(random_state=42)
                X_res, y_res = smote.fit_resample(X_tr, y_tr[:, i])

                model = CatBoostClassifier(
                    iterations=300,
                    learning_rate=0.03,
                    depth=8,
                    l2_leaf_reg=5,
                    border_count=128,
                    verbose=0,
                    random_seed=42,
                    early_stopping_rounds=50
                )
                model.fit(X_res, y_res, eval_set=(X_val, y_val[:, i]), use_best_model=True)
                acc = model.score(X_val, y_val[:, i])
                acc_list.append(acc)

            accs.append(np.mean(acc_list))

        mean_acc = np.mean(accs)
        scores.append(1 - mean_acc)  # PSO minimizes this

    return np.array(scores)

# After PSO optimization, finalize feature mask
best_mask = best_particle > 0.5
selected_features = np.array(features)[best_mask]
print(f"\n✅ Selected features by PSO ({len(selected_features)}):\n{selected_features}")

X_train_final = X_train[:, best_mask]
X_test_final = X_test[:, best_mask]

# Train final models per target using SMOTE
final_models = []
for i in range(y_train.shape[1]):
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X_train_final, y_train[:, i])

    model = CatBoostClassifier(
        iterations=700,
        learning_rate=0.02,
        depth=8,
        l2_leaf_reg=5,
        border_count=128,
        verbose=0,
        random_seed=42,
        early_stopping_rounds=100
    )
    model.fit(X_res, y_res, eval_set=(X_test_final, y_test[:, i]), use_best_model=True)
    final_models.append(model)

# Predict on test data
y_pred = np.column_stack([model.predict(X_test_final) for model in final_models])

test_acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Test Accuracy with SMOTE and selected features: {test_acc:.4f}")

for i, col in enumerate(targets):
    print(f"\nConfusion Matrix for '{col}':")
    print(confusion_matrix(y_test[:, i], y_pred[:, i]))
    print(f"\nClassification Report for '{col}':")
    print(classification_report(y_test[:, i], y_pred[:, i], zero_division=0))
